# OpenDSS

In [1]:
import opendssdirect as odd

In [2]:
odd.__version__

'0.7.0'

In [3]:
odd.run_command("redirect ../data/IEEE13/IEEE13Nodeckt.dss")

''

In [4]:
odd.run_command("Solve")

''

In [5]:
odd.Circuit.SetActiveClass("Vsource")

odd.Circuit.SetActiveElement("source")

0

In [6]:
odd.CktElement.Name()

'Vsource.source'

In [7]:
BASEKV = odd.Vsources.BasekV()

In [8]:
odd.Vsources.BasekV(110)

In [9]:
odd.Vsources.BasekV()

110.0

In [10]:
odd.Vsources.AngleDeg()

30.0

In [11]:
odd.Vsources.AngleDeg(25)

In [12]:
odd.Vsources.AngleDeg()

25.0

In [13]:
# -*- coding: utf-8 -*-
import helics as h

fedinitstring = "--federates=1"
deltat = 0.01

helicsversion = h.helicsGetVersion()

### Create the federate Info object

In [14]:
# Create Federate Info object that describes the federate properties */
print("Creating Federate Info")
fedinfo = h.helicsCreateFederateInfo()

# Set Federate name
print("Setting Federate Info Name")
h.helicsFederateInfoSetCoreName(fedinfo, "OpenDSSFederate")

# Set core type from string
print("Setting Federate Info Core Type")
h.helicsFederateInfoSetCoreTypeFromString(fedinfo, "zmq")

# Federate init string
print("Setting Federate Info Init String")
h.helicsFederateInfoSetCoreInitString(fedinfo, fedinitstring)

# Set the message interval (timedelta) for federate. Note that
# HELICS minimum message time interval is 1 ns and by default
# it uses a time delta of 1 second. What is provided to the
# setTimedelta routine is a multiplier for the default timedelta.

# Set one second message interval
print("Setting Federate Info Time Delta")
h.helicsFederateInfoSetTimeProperty(fedinfo, h.helics_property_time_delta, deltat)

Creating Federate Info
Setting Federate Info Name
Setting Federate Info Core Type
Setting Federate Info Init String
Setting Federate Info Time Delta


### Create the value federate

In [15]:
vfed = h.helicsCreateValueFederate("OpenDSSFederate", fedinfo)
sub = h.helicsFederateRegisterSubscription(vfed, "voltage", "")
sub.set_default(0.0)
pub = h.helicsFederateRegisterGlobalTypePublication(vfed, "load", "complex", "")

### Enter execution

In [16]:
h.helicsFederateEnterExecutingMode(vfed)

### Start simulation

In [17]:
import numpy as np

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

In [18]:
value = 0.0
prevtime = 0

granted_time = -1

for request_time in range(1, 300):

    while granted_time < request_time:
        granted_time = h.helicsFederateRequestTime(vfed, request_time)

    complex_voltage = h.helicsInputGetComplex(sub)
    voltage, angle = complex_voltage.real, complex_voltage.imag

    odd.Circuit.SetActiveClass("Vsource")
    odd.Circuit.SetActiveElement("source")
    odd.Vsources.AngleDeg(angle)
    odd.Vsources.BasekV(voltage * BASEKV)
    print("Source voltage mag at time {}: {} ".format(request_time, voltage))
    print("Source voltage ang at time {}: {} ".format(request_time, angle))
    odd.run_command("Solve")
    
    odd.Circuit.SetActiveClass("Line")
    odd.Circuit.SetActiveElement("650632")
    S = sum([complex(x,y) for x, y in zip(odd.CktElement.Powers()[0:6], odd.CktElement.Powers()[1:6])])
    Pg = S.real
    Qg = S.imag
    print("Active power at time {}: {} ".format(request_time, Pg))
    print("Reactive power at time {}: {} ".format(request_time, Qg))
    
    h.helicsPublicationPublishComplex(pub, Pg / 1000, Qg / 1000)
    
    print()


h.helicsFederateDisconnect(vfed)

h.helicsFederateFree(vfed)
h.helicsCloseLibrary()
print("Federate finalized")

Source voltage mag at time 1: 0.0 
Source voltage ang at time 1: 0.0 
Active power at time 1: -5.608599394082905e-143 
Reactive power at time 1: -7.260114379794892e-143 

Source voltage mag at time 2: 1.0031125352761052 
Source voltage ang at time 2: -4.110753858919751 
Active power at time 2: 4632.3711959333705 
Reactive power at time 2: 4046.1128091804762 

Source voltage mag at time 3: 1.0031125352761052 
Source voltage ang at time 3: -4.110753858919751 
Active power at time 3: 4632.593004603226 
Reactive power at time 3: 4046.3681495060473 

Source voltage mag at time 4: 0.9982575443287671 
Source voltage ang at time 4: -4.419742400565152 
Active power at time 4: 4630.893230921145 
Reactive power at time 4: 4048.476166148458 

Source voltage mag at time 5: 0.998257299996824 
Source voltage ang at time 5: -4.419762054764059 
Active power at time 5: 4630.988219767403 
Reactive power at time 5: 4048.5841567640255 

Source voltage mag at time 6: 0.9982584711931207 
Source voltage ang a